---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
# P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def degree_distrb(graph):
    deg=list(nx.degree(graph).values())
    unique_deg=list(set(deg))
    deg_distrib={i:deg.count(i) for i in unique_deg}
    return deg_distrib

In [4]:
# 'PA'; has high no of degrees------'SW_L';doesnt follow power law;low average shortest distance; high avg clustering index------
#---'SW_H';doesnt follow power law;low average shortest distance; low avg clustering index 
#
def graph_identification():
    
    # Your Code Here
#     for i in P1_Graphs:
#         print(nx.average_shortest_path_length(i),nx.average_clustering(i),len(degree_distrb(i)))

    return ['PA','SW_L','SW_L','PA','SW_H']# Your Answer Here
# graph_identification()

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [5]:
G = nx.read_gpickle('email_prediction.txt')


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [6]:
def salary_predictions():
    from sklearn.svm import SVC
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import GradientBoostingClassifier

    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    from sklearn.metrics import roc_auc_score
    from sklearn.model_selection import train_test_split
    from sklearn.model_selection import GridSearchCV
    from sklearn.metrics import classification_report
        ##checking nan

    def mangmnetsalry(node):
        n=node[1]['ManagementSalary']
        return n



    df=pd.DataFrame(G.nodes())
    df['degree_centrality']=list(nx.degree_centrality(G).values())
    df['closeness_centrality']=list(nx.closeness_centrality(G,normalized=True).values())
    df['between_centrality']=list(nx.betweenness_centrality(G,normalized=True,endpoints=False).values())
    df['pagerank']=list(nx.pagerank(G).values())
    # df['hubs']=list(nx.hits(G)[0].values())
    # df['authority']=list(nx.hits(G)[1].values())
    df['clustering']=list(nx.clustering(G).values())
    df['degree']=list(nx.degree(G).values())
    df['management']=[mangmnetsalry(node) for node in G.nodes(data=True)]
    trainset=df.loc[~df['management'].isnull(),:]
    predset=df.loc[df['management'].isnull(),:]
    predset.drop(['management'],inplace=True,axis=1)
    X_train,X_test,y_train,y_test=train_test_split(trainset.iloc[:,:7],trainset.iloc[:,7],random_state=0,train_size=0.8)

    clf_lr=LogisticRegression(penalty='l2')
    param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }
    clf_lr_gs=GridSearchCV(clf_lr,param_grid=param_grid,scoring='roc_auc')
    clf_lr_gs.fit(X_train,y_train)
    print('Grid best parameter (max. auc): ', clf_lr_gs.best_params_)
    print('Train set AUC(SVC): ', roc_auc_score(y_train, clf_lr_gs.decision_function(X_train)))
    # clf_lr=LogisticRegression(penalty='l2',C=100).fit(X_train,y_train)
    pred_lr=clf_lr_gs.decision_function(X_test)
    print('Test set AUC(LogisticReg): ', roc_auc_score(y_test, pred_lr))
    print('Test set F1(LogisticReg): ', f1_score(y_test, clf_lr_gs.predict(X_test)))
    print('\n\n',classification_report(y_test, clf_lr_gs.predict(X_test)))
    pred_lr_unknown=clf_lr_gs.predict_proba(predset)
    pred_lr_unknown=np.array([i[1] for i in pred_lr_unknown]).reshape((-1,1))

    clf_svc=SVC(probability=True)
    param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],'gamma': [0.0001, 0.001,0.01,0.1,1,10],'kernel': ['rbf'] }
    clf_svc_gs=GridSearchCV(clf_svc,param_grid=param_grid,scoring='roc_auc')
    clf_svc_gs.fit(X_train,y_train)
    print('Grid best parameter (max. auc): ', clf_svc_gs.best_params_)
    print('Train set AUC(SVC): ', roc_auc_score(y_train, clf_svc_gs.decision_function(X_train)))
    pred_svc=clf_svc_gs.decision_function(X_test)
    print('Test set AUC(SVC): ', roc_auc_score(y_test, pred_svc))
    print('Test set F1(SVC): ', f1_score(y_test, clf_svc_gs.predict(X_test)))
    print('\n\n',classification_report(y_test, clf_svc_gs.predict(X_test)))
    pred_svc_unknown=clf_svc_gs.predict_proba(predset)
    pred_svc_unknown=np.array([i[1] for i in pred_svc_unknown]).reshape((-1,1))

    clf_gbm=GradientBoostingClassifier()
    param_grid = gb_grid_params = {'n_estimators':[100],
                                   'learning_rate': [0.1, 0.01],
                  'max_depth': [4, 8],
                  'min_samples_leaf': [20, 50,100],
                  'max_features': [1.0, 0.5, 0.1] 
                                  }
    clf_gbm_gs=GridSearchCV(clf_gbm,param_grid=param_grid,scoring='roc_auc')
    clf_gbm_gs.fit(X_train,y_train)
    print('Grid best parameter (max. auc): ', clf_gbm_gs.best_params_)
    print('Train set AUC(gbm): ', roc_auc_score(y_train, clf_gbm_gs.decision_function(X_train)))
    pred_gbm=clf_gbm_gs.decision_function(X_test)
    print('Test set AUC(gbm): ', roc_auc_score(y_test, pred_gbm))
    print('Test set F1(gbm): ', f1_score(y_test, clf_gbm_gs.predict(X_test)))
    print('\n\n',classification_report(y_test, clf_gbm_gs.predict(X_test)))
    pred_gbm_unknown=clf_gbm_gs.predict_proba(predset)
    pred_gbm_unknown=np.array([i[1] for i in pred_gbm_unknown]).reshape((-1,1))

    ensemble=list(np.mean(np.array(list(zip(pred_gbm_unknown,pred_lr_unknown,pred_svc_unknown))).reshape(-1,3),axis=1))
    lst=pd.Series(ensemble,index=predset.index)
    return lst
salary_predictions()

C:\Users\rai16\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\rai16\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Grid best parameter (max. auc):  {'C': 100}
Train set AUC(SVC):  0.8799335617149382
Test set AUC(LogisticReg):  0.8628608923884514
Test set F1(LogisticReg):  0.42424242424242425


              precision    recall  f1-score   support

        0.0       0.88      0.98      0.93       127
        1.0       0.78      0.29      0.42        24

avg / total       0.86      0.87      0.85       151

Grid best parameter (max. auc):  {'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
Train set AUC(SVC):  0.9888093013599086
Test set AUC(SVC):  0.7352362204724409
Test set F1(SVC):  0.0


              precision    recall  f1-score   support

        0.0       0.84      1.00      0.91       127
        1.0       0.00      0.00      0.00        24

avg / total       0.71      0.84      0.77       151



C:\Users\rai16\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\rai16\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Grid best parameter (max. auc):  {'learning_rate': 0.1, 'max_depth': 8, 'max_features': 0.5, 'min_samples_leaf': 20, 'n_estimators': 100}
Train set AUC(gbm):  1.0
Test set AUC(gbm):  0.9537401574803149
Test set F1(gbm):  0.631578947368421


              precision    recall  f1-score   support

        0.0       0.91      0.98      0.95       127
        1.0       0.86      0.50      0.63        24

avg / total       0.90      0.91      0.90       151



1       0.076579
2       0.619516
5       0.796049
8       0.120460
14      0.139552
18      0.217085
27      0.414503
30      0.630290
31      0.472112
34      0.083841
37      0.076175
40      0.359591
45      0.326709
54      0.189220
55      0.143214
60      0.147440
62      0.745719
65      0.519144
77      0.044026
79      0.087085
97      0.056568
101     0.035885
103     0.343386
108     0.084458
113     0.201839
122     0.033762
141     0.447383
142     0.750169
144     0.076332
145     0.365018
          ...   
913     0.037632
914     0.049263
915     0.029377
918     0.103625
923     0.038422
926     0.079147
931     0.038781
934     0.028537
939     0.028530
944     0.025814
945     0.041504
947     0.117092
950     0.095343
951     0.043905
953     0.036566
959     0.028299
962     0.027365
963     0.134696
968     0.090589
969     0.085650
974     0.065196
984     0.029957
987     0.090334
989     0.094454
991     0.096185
992     0.035386
994     0.034032
996     0.0321

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [8]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [11]:
def new_connections_predictions():
    
    from sklearn.svm import SVC
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import GradientBoostingClassifier

    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    from sklearn.metrics import roc_auc_score
    from sklearn.model_selection import train_test_split
    from sklearn.model_selection import GridSearchCV
    from sklearn.metrics import classification_report

    future_connections['node1']=future_connections.index.map(lambda x:x[0])
    future_connections['node2']=future_connections.index.map(lambda x:x[1])
    jacard_coef={i[:2]:i[2] for i in nx.jaccard_coefficient(G)}
    resource_coef={i[:2]:i[2] for i in nx.resource_allocation_index(G)}
    pref_attachment={i[:2]:i[2] for i in nx.preferential_attachment(G)}
    future_connections['jacard_coef']=future_connections.index.map(lambda x :jacard_coef[x])
    future_connections['resource_coef']=future_connections.index.map(lambda x :resource_coef[x])
    future_connections['pref_attachment']=future_connections.index.map(lambda x :pref_attachment[x])

    trainset=future_connections.loc[~future_connections['Future Connection'].isnull(),:]
    predset=future_connections.loc[future_connections['Future Connection'].isnull(),:]
    predset.drop(['Future Connection'],inplace=True,axis=1)
    X_train,X_test,y_train,y_test=train_test_split(trainset.iloc[:,1:],trainset.iloc[:,0],random_state=0,train_size=0.8)


    clf_lr=LogisticRegression(penalty='l2')
    param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }
    clf_lr_gs=GridSearchCV(clf_lr,param_grid=param_grid,scoring='roc_auc')
    clf_lr_gs.fit(X_train,y_train)
    print('Grid best parameter (max. auc): ', clf_lr_gs.best_params_)
    print('Train set AUC(SVC): ', roc_auc_score(y_train, clf_lr_gs.decision_function(X_train)))
    # clf_lr=LogisticRegression(penalty='l2',C=100).fit(X_train,y_train)
    pred_lr=clf_lr_gs.decision_function(X_test)
    print('Test set AUC(LogisticReg): ', roc_auc_score(y_test, pred_lr))
    print('Test set F1(LogisticReg): ', f1_score(y_test, clf_lr_gs.predict(X_test)))
    print('\n\n',classification_report(y_test, clf_lr_gs.predict(X_test)))
    pred_lr_unknown=clf_lr_gs.predict_proba(predset)
    pred_lr_unknown=np.array([i[1] for i in pred_lr_unknown]).reshape((-1,1))

#     clf_svc=SVC(probability=True)
#     param_grid = {'C': [ 0.01, 0.1, 1, 10, 100],'gamma': [0.0001, 0.001],'kernel': ['rbf'] }
#     clf_svc_gs=GridSearchCV(clf_svc,param_grid=param_grid,scoring='roc_auc')
#     clf_svc_gs.fit(X_train,y_train)
#     print('Grid best parameter (max. auc): ', clf_svc_gs.best_params_)
#     print('Train set AUC(SVC): ', roc_auc_score(y_train, clf_svc_gs.decision_function(X_train)))
#     pred_svc=clf_svc_gs.decision_function(X_test)
#     print('Test set AUC(SVC): ', roc_auc_score(y_test, pred_svc))
#     print('Test set F1(SVC): ', f1_score(y_test, clf_svc_gs.predict(X_test)))
#     print('\n\n',classification_report(y_test, clf_svc_gs.predict(X_test)))
#     pred_svc_unknown=clf_svc_gs.predict_proba(predset)
#     pred_svc_unknown=np.array([i[1] for i in pred_svc_unknown]).reshape((-1,1))

#     clf_gbm=GradientBoostingClassifier()
#     param_grid = gb_grid_params = {'n_estimators':[100],
#                                    'learning_rate': [0.1, 0.01],
#                   'max_depth': [4, 8],
#                   'max_features': [1.0, 0.5, 0.1] 
#                                   }
#     clf_gbm_gs=GridSearchCV(clf_gbm,param_grid=param_grid,scoring='roc_auc')
#     clf_gbm_gs.fit(X_train,y_train)
#     print('Grid best parameter (max. auc): ', clf_gbm_gs.best_params_)
#     print('Train set AUC(gbm): ', roc_auc_score(y_train, clf_gbm_gs.decision_function(X_train)))
#     pred_gbm=clf_gbm_gs.decision_function(X_test)
#     print('Test set AUC(gbm): ', roc_auc_score(y_test, pred_gbm))
#     print('Test set F1(gbm): ', f1_score(y_test, clf_gbm_gs.predict(X_test)))
#     print('\n\n',classification_report(y_test, clf_gbm_gs.predict(X_test)))
#     pred_gbm_unknown=clf_gbm_gs.predict_proba(predset)
#     pred_gbm_unknown=np.array([i[1] for i in pred_gbm_unknown]).reshape((-1,1))
    ensemble=list(pred_lr_unknown)
#     ensemble=list(np.mean(np.array(list(zip(pred_gbm_unknown,pred_lr_unknown,pred_svc_unknown))).reshape(-1,3),axis=1))
    lst=pd.Series(ensemble,index=predset.index.values)
    return lst
new_connections_predictions()

C:\Users\rai16\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\rai16\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Grid best parameter (max. auc):  {'C': 0.1}
Train set AUC(SVC):  0.8957840256409394
Test set AUC(LogisticReg):  0.8970674390282364
Test set F1(LogisticReg):  0.5201342281879194


              precision    recall  f1-score   support

        0.0       0.95      0.99      0.97     67447
        1.0       0.86      0.37      0.52      5820

avg / total       0.94      0.95      0.94     73267



(107, 348)    [0.057587478321758465]
(542, 751)    [0.024443558701801836]
(20, 426)       [0.3715691987496757]
(50, 989)      [0.01894458767588255]
(942, 986)      [0.0194285024990055]
(324, 857)    [0.021633767728580975]
(13, 710)       [0.2200884201554855]
(19, 271)       [0.3313747799503839]
(319, 878)    [0.021017082187831992]
(659, 707)     [0.02549637447455735]
(49, 843)       [0.0212372398588715]
(208, 893)    [0.021068527201408626]
(377, 469)     [0.07572411013810333]
(405, 999)    [0.023128711260355497]
(129, 740)     [0.03170393147121033]
(292, 618)     [0.05477520816701523]
(239, 689)    [0.024663851923544606]
(359, 373)     [0.05627473404526143]
(53, 523)      [0.05557492816539212]
(276, 984)    [0.019002301149543026]
(202, 997)     [0.01863355581129446]
(604, 619)     [0.09284649700747868]
(270, 911)    [0.020344052325536297]
(261, 481)     [0.11011181391529042]
(200, 450)      [0.4831273924341433]
(213, 634)     [0.02680219914850114]
(644, 735)     [0.06687135586612043]
(